Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

algoritmo de classificação utilizado no titanic
wording bading / baggle of words/ df-tf (atribui pesos as palvras)
feater extraction -> transformar o texto em uma coluna de banco de dados

1)Limpeza das palavras: tirar as stop words, lematização, Bag-of-words

## 1) Vizualização dos dados

In [35]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [53]:
from sklearn.model_selection import GridSearchCV

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
from nltk.corpus import stopwords

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
tweets = train_df['text']

## 2) Minimização dos ruídos

### Text

In [25]:
sample_tweets_train=train_df['text']
sample_tweets_test=test_df['text']

In [26]:
def limpeza(tweets):
    tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
    tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
    tweets = tweets.str.replace(r'[0-9]','')
    tweets = tweets.replace("\\x89Û_",'')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÒ','')
    tweets = tweets.replace('\x89Û','')
    tweets = tweets.replace('\x89Û÷','')
    tweets = tweets.replace('\x89ûï','')
    #retirada de símbolos não alfanuméticos 
    tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")
    tweets = tweets.str.lower()
    return tweets

In [27]:
sample_tweets_train = limpeza(sample_tweets_train)
sample_tweets_test = limpeza(sample_tweets_test)

C:\Users\yfrom\AppData\Local\Temp\ipykernel_4940\1236119256.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_4940\1236119256.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_4940\1236119256.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'[0-9]','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_4940\1236119256.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")


In [18]:
stop_words = set(stopwords.words("english"))

In [28]:
def lematizar(tweets):
    sample_tweets= []
    for tweet in tweets:
        filtered_list = []
        words_lemmatized = []
        words = []
        words = word_tokenize(tweet) 
        words_lemmatized = [lemmatizer.lemmatize(word) for word in words] 
        for word in words_lemmatized:
            if word not in stop_words:
                filtered_list.append(word)
        sample_tweets.append(filtered_list)
    return sample_tweets    

In [29]:
sample_tweets_train = lematizar(sample_tweets_train)
sample_tweets_test = lematizar(sample_tweets_test)

In [21]:
train_df['words']= sample_tweets_train
test_df['words']= sample_tweets_test

In [22]:
train_df.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[deed, reason, earthquake, may, allah, forgive..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[resident, asked, 'shelter, place, ', notified..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfire, evacuation, order,..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [30]:
test_df = test_df.drop(columns=['location', 'keyword','id','text'])
train_df = train_df.drop(columns=['location', 'keyword','id','text'])

In [31]:
train_df['text'] = train_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))
test_df['text'] = test_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [32]:
train_df.head()

,target,words,text
0,1,"[deed, reason, earthquake, may, allah, forgive...",deed reason earthquake may allah forgive u
1,1,"[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,1,"[resident, asked, 'shelter, place, ', notified...",resident asked 'shelter place ' notified offic...
3,1,"[people, receive, wildfire, evacuation, order,...",people receive wildfire evacuation order calif...
4,1,"[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfire pour...


## Bag-of-words
 Definição: Bag-of-words model is a way of representing text data when modeling text with machine learning algorithms. Machine learning algorithms cannot work with raw text directly; the text must be converted into well defined fixed-length(vector) numbers.

Formação da Bag-of-word para treino e teste com count vectorizer

In [38]:
treino = train_df.head(6090)
teste = train_df.tail(1523)

In [46]:
target_treino= treino['target']
traget_teste= teste['target']

In [47]:
Vectorizer = CountVectorizer()
count_treino_cv = Vectorizer.fit_transform(treino['text'])
count_teste_cv = Vectorizer.transform(teste['text'])

Formação da Bag-of-word para treino e teste com tf-idf

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
tfidf_vectorizer = TfidfVectorizer()
count_treino_tfidf = tfidf_vectorizer.fit_transform(treino['text'])
count_teste_tfidf = tfidf_vectorizer.transform(teste['text'])

word2vec

In [ ]:
#fazer aqui o word2vec

## Modelos de classificação

In [40]:
def metricas(y_test,y_predicted):
    precision = precision_score(y_test, y_predicted, pos_label=None,average='weighted')             
    recall = recall_score(y_test, y_predicted, pos_label=None,average='weighted')
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1


### Regressão Logística

GridSearch

In [79]:
parameters = [{
'C': [1,2,3,30,50,],
'max_iter': [1000,100000],
'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
'class_weight':['dict','balanceado'],
'multi_class': ['auto', 'ovr', 'multinomial']
}]

In [80]:
clf_tfidf = GridSearchCV(
        LogisticRegression(), parameters, scoring='accuracy')
clf_tfidf.fit(count_treino_tfidf, target_treino)

c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
700 fits failed out of a total of 1500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py", line 1158, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "c:\Users\yfrom\AppData\Local\Programs\Pyth

GridSearchCV(estimator=LogisticRegression(),
             param_grid=[{'C': [1, 2, 3, 30, 50],
                          'class_weight': ['dict', 'balanceado'],
                          'max_iter': [1000, 100000],
                          'multi_class': ['auto', 'ovr', 'multinomial'],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']}],
             scoring='accuracy')

In [82]:
print(clf_tfidf.best_params_)
#Para o Count Vector:
#{'C': 1, 'class_weight': 'balanceado', 'max_iter': 100000, 'multi_class': 'auto', 'solver': 'sag'}

{'C': 1, 'class_weight': 'dict', 'max_iter': 1000, 'multi_class': 'auto', 'solver': 'newton-cg'}


In [59]:
clf_cv = LogisticRegression(C=1, class_weight='balanced', solver='sag',multi_class='auto', n_jobs=-1, random_state=40,max_iter= 100000)
clf_cv.fit(count_treino_cv, target_treino)
Predição_target_cv = clf_cv.predict(count_teste_cv)

In [60]:
print(metricas(traget_teste, Predição_target_cv))

(0.7879185817465528, 0.7896749023950892, 0.7879185817465528, 0.7865201956024487)


In [83]:
clf_tfidf = LogisticRegression(C=1, class_weight='dict', solver='newton-cg',multi_class='auto', n_jobs=-1, random_state=40,max_iter= 1000)
clf_tfidf.fit(count_treino_tfidf, target_treino)
Predição_target_tfidf = clf_tfidf.predict(count_teste_tfidf)

In [84]:
print(metricas(traget_teste, Predição_target_tfidf))
#Sem parâmetros ideias:
#(0.7912015758371634, 0.7911097527493921, 0.7912015758371634, 0.7908037193499462)

(0.7925147734734077, 0.8075907785931117, 0.7925147734734077, 0.7875804286920929)


SVM

In [85]:
parameters = [{
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
'C': [1,2,3,300,500],
'max_iter': [1000,100000],
'gamma':['scale', 'auto'],
'class_weight':['dict' ,'balanced']
}]

In [86]:
clf = GridSearchCV(
        SVC(), parameters, scoring='accuracy'
    )
clf.fit(count_treino_tfidf, target_treino)

c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\yfrom\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 2, 3, 300, 500],
                          'class_weight': ['dict', 'balanced'],
                          'gamma': ['scale', 'auto'],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                          'max_iter': [1000, 100000]}],
             scoring='accuracy')

In [87]:
print(clf.best_params_)

{'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 100000}


In [73]:
clf = SVC(kernel='rbf',C=1,class_weight='balanced',gamma='scale', max_iter=100000) 
clf.fit(count_treino_cv, target_treino)
Predição_target_SVM_cv = clf.predict(count_teste_cv)

In [74]:
print(metricas(traget_teste, Predição_target_SVM_cv))
#antes do grid search: (0.7544320420223244, 0.7607494234790165, 0.7544320420223244, 0.7505317206525575)

(0.7820091923834537, 0.7990065430270551, 0.7820091923834537, 0.7761385531145388)


In [75]:
clf = SVC(kernel='rbf',C=1,class_weight='balanced',gamma='scale', max_iter=100000) 
clf.fit(count_treino_tfidf, target_treino)
Predição_target_SVM_tfidf= clf.predict(count_teste_tfidf)

In [77]:
print(metricas(traget_teste, Predição_target_SVM_tfidf))

(0.7820091923834537, 0.7866863652510794, 0.7820091923834537, 0.7794820049168832)


### Dicas da Maria:
- grid search
- Lineares:
    - SVM
    - naivebayes
    - random forest
- Outra abordargem (neural);
    - word2vec -> cnn 
    - wor2vec -> outra coisa ai com neuronio